# Xception Network

#### Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

#### Entry flow

In [46]:
def entry_flow(inputs):
    
    x = keras.layers.Conv2D(32, 3, strides=2, padding='same')(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)

    x = keras.layers.Conv2D(64, 3, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)

    previous_block_activation = x

    for size in [128, 256, 728]:

        x = keras.layers.Activation('relu')(x)
        x = keras.layers.SeparableConv2D(size, 3, padding='same')(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation('relu')(x)
        x = keras.layers.SeparableConv2D(size, 3, padding='same')(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.MaxPooling2D(3, strides=2, padding='same')(x)

        residual = keras.layers.Conv2D(size, 1, strides=2, padding='same')(previous_block_activation)

        x = keras.layers.Add([x, residual])
        previous_block_activation = x

    return x

#### Middle Flow

In [48]:
def middle_flow(x, num_blocks=8):

    previous_block_activation = x

    for _ in range(num_blocks):

        x = keras.layers.Activation('relu')(x)
        x = keras.layers.SeparableConv2D(728, 3, padding='same')(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation('relu')(x)
        x = keras.layers.SeparableConv2D(728, 3, padding='same')(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation('relu')(x)
        x = keras.layers.SeparableConv2D(728, 3, padding='same')(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Add([x, previous_block_activation])
        previous_block_activation = x

    return x

#### Exit Flow

In [50]:
def exit_flow(x):

    previous_block_activation = x

    x = keras.layers.Activation('relu')(x)
    x = keras.layers.SeparableConv2D(728, 3, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)

    x = keras.layers.Activation('relu')(x)
    x = keras.layers.SeparableConv2D(1024, 3, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)

    x = keras.layers.MaxPooling2D(3, strides=2, padding='same')(x)

    residual = keras.layers.Conv2D(1024, 1, strides=2, padding='same')(previous_block_activation)
    x = keras.layers.Add([x, residual])

    x = keras.layers.Activation('relu')(x)
    x = keras.layers.SeparableConv2D(728, 3, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)

    x = keras.layers.Activation('relu')(x)
    x = keras.layers.SeparableConv2D(1024, 3, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)

    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(1, activation='linear')(x)

    return x

In [36]:
inputs = keras.layers.Input(shape=(128, 128, 1))

x = entry_flow(inputs)
x = middle_flow(x)
outputs = exit_flow(x)

xception = keras.Model(inputs, outputs)